<a href="https://colab.research.google.com/github/maria-chiara-mar/stocks_autopilot/blob/main/FINAL_Stocks_Autopilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script uses the GitHub library EdgarTools to extract companies' financial information from the US SEC database EDGAR, transforms them into data-frames (**Step 1**), and plugs the info in the corresponding location in the company google sheet (**Step 4**). The Google Sheet is used for modeling future cash flows and calculating the price per share of different companies. It finds the correct location in the google sheet with the help of a dictionary, created in the google sheet (**Step 2**). Optionaly, you can export the transformed data into the google sheet (**Step 3**).


**Step 0**
Installs the package, and imports all relevant libraries, functions, authentifications.

In [ ]:
# Insalls GitHub library EdgarTools
!pip install edgartools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
# EdgarTools imports
from edgar import *
from edgar.xbrl import XBRLS
set_identity("kindemilvc@gmail.com") #email address here (auth for SEC)

# Pandas / Datetime import
import pandas as pd
from datetime import datetime

# Gspread for google sheet manipulation
import gspread
from gspread_dataframe import set_with_dataframe

# Google authentification to access google sheets
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
google_sheet_name = "Stonks 3.0" # <-- put here name of the google sheet (relevant for Steps 2, 3, 4 with google sheet interactions)



**Step 1**
The three financial statements are called, transformed, unpivoted, and appended to one flat dataframe
- Cashflow: extract stitched_statement to retrieve information on several years;
- Balance Sheet: same as above;
- Income Statement: extract the full statement to retrieve very precise and highly granular information, but atm we are only able to export last 3 years;

In [ ]:
def get_company_data(ticker, years):

  c = Company(ticker)
  tenk = c.get_filings(form="10-K") # get annual reports only

  # for income statement - has detailed revenue breakdowns
  latest = tenk.latest() # limited to last 3 years
  xb = latest.xbrl()
  inc = xb.statements.income_statement()

  # for balance sheet and cash flow
  latest2 = tenk.latest(years) # get x years of historical data
  xb_years = XBRLS.from_filings(latest2)
  stitched_statements = xb_years.statements # merge historical years
  bs = xb_years.statements.balance_sheet()
  cf = xb_years.statements.cashflow_statement()

  # transformed into dataframes
  df_inc = inc.to_dataframe()
  df_bs = bs.to_dataframe()
  df_cf = cf.to_dataframe()

  # append the three stratements and drop unnecessary columns
  df = pd.concat([df_inc, df_bs, df_cf])
  df = df.drop(columns = ["level","abstract","dimension","balance","weight","preferred_sign","parent_concept"])
  df['ticker'] = ticker # add ticker column

  print("Extracted company data")

  return df

def unpivot_df(df):
  # unpivot date and value columns to get flat file format
  columns_tokeep = ["ticker", 'concept', 'label']
  columns_tomelt = [col for col in df.columns if col not in columns_tokeep]
  df = pd.melt(df, id_vars=columns_tokeep, value_vars=columns_tomelt, var_name='date', value_name='value')

  print("Unpivoted company data")

  return df


**Step 2**
Merge your unpivoted df with the google sheet "Dictionary", which contains the financial terms we use in our workframe. This step is necessary to plug-in the extracted data at the correct location.

In [ ]:
def import_dictionary(df_financials):
  # find dictionary
  sh = gc.open(google_sheet_name) # <-- put here name of the google sheet (relevant for Steps 2, 3, 4 with google sheet interactions)
  worksheet = sh.worksheet('Dictionary') # sh. worksheet was already defined and opened in Step 0

  # gets all values from the worksheet
  data_list = worksheet.get_all_values()

  # create a dataframe from the list of lists, using the first row as headers
  df_dictionary = pd.DataFrame(data_list[1:], columns=data_list[0])

  # gets rid of other columns if accidentaly imported
  columns_tokeep = ["ticker", "concept", "label", "name"]
  df_dictionary = df_dictionary[columns_tokeep]

  # joins df_dictionary and combined financial statements (inner merge)
  common_keys = ['ticker', 'concept', 'label']
  df_final = pd.merge(df_financials, df_dictionary, on=common_keys, how='inner')

  print("Merged with dictionary")

  return df_final

**Step 3 - OPTIONAL**
Export your final DF to the Statements Database sheet, which will serve as your database for further analysis. Eliminate all manual steps by using the gspread automation.

In [ ]:
def export_to_gsheet(df_final):
  # opens the target google sheet (must already exist)
  sh = gc.open(google_sheet_name) # <-- put here name of the google sheet (relevant for Steps 2, 3, 4 with google sheet interactions)
  target_sheet_name = 'Edgar'
  worksheet = sh.worksheet(target_sheet_name) # sh. worksheet was already defined and opened in Step 0

  # clears existing content in the target sheet before uploading new data
  worksheet.clear()

  # uploads the df_final to the selected worksheet
  set_with_dataframe(worksheet, df_final, include_index=False, include_column_header=True)

  print("Exported to Google Sheet")

  return

**Step 4**
Plugs the extracted financials to the correct location in the corresponding company google sheet.

In [ ]:
def plugging_in_gsheet(df_final):
  # opens the company google sheet
  sh = gc.open(google_sheet_name) # <-- put here name of the google sheet (relevant for Steps 2, 3, 4 with google sheet interactions)
  ticker = df_final.iloc[0]["ticker"]
  worksheet = sh.worksheet(ticker) # sh. worksheet was already defined and opened in Step 0

  # defines sheet_ranges where to find row-/column index outside of the loop
  range_date = worksheet.get("K3:ZZ4")
  range_row = worksheet.get("C:C")

  # loops through each row of the dataframe from step 2 - each row is one data point from the company financials
  for rows in df_final.iterrows():

    # FIND DATE IN LIST OF DATES FROM GS
    hist_horizon = int(range_date[1][0].strip("()")) # from google sheets extracts how many historical years we have
    date_list = []
    count = 0
    for row in range_date:
      if count > hist_horizon: break
      for cell in row:
        #cell = datetime.strptime(cell,"%b/%Y")
        date_list.append(cell)
        count = count +1
        if count > hist_horizon:
          break
    # GET DATE (COLUMN) INDEX
    df_date = rows[1]["date"]
    df_date = datetime.strptime(df_date,"%Y-%m-%d")
    date_index = date_list.index(df_date.strftime("%b/%Y")) + 11 # column K-column C

    # GET ROW INDEX
    name = [rows[1]["name"]]
    row_index = 0
    for row in range_row:
      row_index = row_index + 1
      if row == name:
        break

    # transforms value to millions
    value = rows[1]["value"]/10**6

    # plugs in each row value into the correct date/column, and row index
    worksheet.update_cell(row_index, date_index, value)

  print("Inserted data points into google sheet")

  return


**Execution** Calls the relevant functions for ticker x for y years

In [ ]:
ticker = "HUBS"
years = 3

# Step 1 - gets company data
df_financials = get_company_data(ticker, years) # execute only this function to get pivoted data - good for dictionary preparation
df_financials_flat = unpivot_df(df_financials) # just unpivots the data

# Step 2 - merges with dictionary
df_final = import_dictionary(df_financials_flat)

# Steps 3 (optional) - export df to google sheet
export_to_gsheet(df_financials_flat)

# Step 4 - plugs in data into company google sheet
plugging_in_gsheet(df_final)

Extracted company data
Unpivoted company data
Merged with dictionary
Exported to Google Sheet
Inserted data points into google sheet
